In [0]:
%matplotlib inline
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
#import seaborn as sns; sns.set()
import os
import math
import random
from collections import Counter
from skimage import feature
import cv2

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC, LinearSVC
from sklearn.cluster import MiniBatchKMeans
from sklearn.externals import joblib
from skimage.feature import hog

In [2]:
from google.colab import files
def getLocalFiles():
    _files = files.upload()
    if len(_files) >0:
       for k,v in _files.items():
         open(k,'wb').write(v)
getLocalFiles()

Saving 101_ObjectCategories.tar.gz to 101_ObjectCategories.tar (2).gz


In [0]:
categories = ['airplanes', 'Faces_easy', 'Leopards', 'Motorbikes', 'watch']
#root = '101_ObjectCategories'
import tarfile
root = tarfile.open("101_ObjectCategories.tar.gz")
root.extractall()
root.close()

In [0]:
def get_feature_space(images, feature_type):
    features = []
    if feature_type == 'sift':
        sift = cv2.xfeatures2d.SIFT_create()
        for image in images:
            kp, des = sift.detectAndCompute(image, None)
            features.extend(des.tolist())
    elif feature_type == 'hog':
        for image in images:
            
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            res = cv2.resize(gray, (101, 128), interpolation=cv2.INTER_AREA)
            des = hog(res, block_norm='L2-Hys')
            features.extend(des.reshape(-1, 36).tolist())
        
            
    return features

"""
Only splitting into training and testing sets because I use cross-validation.
You can look at older commits in the repository for examples 

"""
def load_dataset():
    X_train, X_test, y_train, y_test = [], [], [], []
    for category in categories:
        X, y = [], []
        for file in root:
            file = os.fsdecode(file)
            if file.endswith(".jpg"):
                filepath = os.path.join(root, category, file)
                img = cv2.imread(filepath)
                X.append(img)
                y.append(category)

        X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.25, shuffle=True)

        X_train.extend(X_tr)
        X_test.extend(X_te)
        y_train.extend(y_tr)
        y_test.extend(y_te)
    return X_train, X_test, y_train, y_test


def to_sift_features(images):
    sift = cv2.xfeatures2d.SIFT_create()
    return [sift.detectAndCompute(image, None)[1].tolist() for image in images]

def to_hog_features(images):
    gray = [cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in images]
    res = [cv2.resize(g, (101, 128), interpolation=cv2.INTER_AREA) for g in gray]
    return [hog(r, block_norm='L2-Hys').reshape(-1, 36).tolist() for r in res]
    


def to_BOW_features(features, codebook):
    bags_of_words = [codebook.predict(feature) for feature in features]
    hists = [np.histogram(bag, bins=codebook.n_clusters)[0] for bag in bags_of_words]
    return hists

In [0]:
X_train, X_test, y_train, y_test = load_dataset()

# Encode labels
le = LabelEncoder()
le.fit(categories)
y_train = le.transform(y_train)
y_test = le.transform(y_test)

In [0]:
feature_space_sift = get_feature_space(X_train, 'sift')

# Convert training and test set to SIFT
X_train_sift = to_sift_features(X_train)
X_test_sift = to_sift_features(X_test)

In [0]:
X_train = np.asarray(X_train_sift)
X_test = np.asarray(X_test_sift)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [0]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1853,) (619,) (1853,) (619,)


In [0]:
km = MiniBatchKMeans(20).fit(feature_space_sift)
X_train_bow = to_BOW_features(X_train_sift, km)
X_test_bow = to_BOW_features(X_test_sift,km)

In [0]:
x_train = np.array(X_train_bow)
x_test = np.array(X_test_bow)
y_train = keras.utils.to_categorical(y_train, 5)
y_test = keras.utils.to_categorical(y_test, 5)

In [0]:
x_train.shape

(1853, 20)

In [0]:
import tensorflow as tf
import os
from tensorflow import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [0]:
batch_size = 10
num_classes = 5
epochs = 10
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_bow_model.h5'

In [0]:
model = Sequential()
model.add(Dense(20,activation='relu',input_shape=(20,)))
model.add(Dropout(0.2))
model.add(Dense(20,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(5,activation='softmax'))
model.compile(optimizer = 'adam',loss= 'categorical_crossentropy', metrics = ['accuracy'])

In [0]:
model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              )

Train on 1853 samples, validate on 619 samples
Epoch 1/10
1853/1853 [==============================] - 1s 476us/step - loss: 3.3133 - acc: 0.2806 - val_loss: 1.3887 - val_acc: 0.5315
Epoch 2/10
1853/1853 [==============================] - 0s 255us/step - loss: 1.4576 - acc: 0.4808 - val_loss: 1.1788 - val_acc: 0.6107
Epoch 3/10
1853/1853 [==============================] - 0s 261us/step - loss: 1.2909 - acc: 0.5472 - val_loss: 1.1027 - val_acc: 0.6511
Epoch 4/10
1853/1853 [==============================] - 1s 276us/step - loss: 1.1432 - acc: 0.5855 - val_loss: 1.0021 - val_acc: 0.6527
Epoch 5/10
1853/1853 [==============================] - 1s 274us/step - loss: 1.1437 - acc: 0.6033 - val_loss: 0.9837 - val_acc: 0.6834
Epoch 6/10
1853/1853 [==============================] - 0s 257us/step - loss: 1.0359 - acc: 0.6363 - val_loss: 0.9494 - val_acc: 0.6914
Epoch 7/10
1853/1853 [==============================] - 1s 279us/step - loss: 0.9848 - acc: 0.6395 - val_loss: 0.9015 - val_acc: 0.7124
E

In [0]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

619/619 [==============================] - 0s 55us/step
Test loss: 0.839272064423
Test accuracy: 0.746365105201
